# PPO Base Implementation
This will be the baseline implementation for comparing with the other methods.

Python Requirements:

In [1]:
import torch

In [ ]:
LEARNING_RATE = 1e-4
GAMMA = 0.99
EPOCHS = 10
CLIP_EPSILON = 0.2
BATCH_SIZE = 64